In [93]:
from glob import glob
import pickle
import torch
from utils import initialize
import os
import numpy as np
import pandas as pd

csv_feature_dict, label_encoder, label_decoder = initialize()

In [94]:
def process_result(result):
    logits = [o for batch in result[0] for o in batch['logits']]
    processed = torch.stack(logits, axis=0)
    # processed = torch.max(processed, axis=1, keepdim=True)
    return processed

In [95]:
result_list = []

for file in glob("results/*"):
    with open(file, 'rb') as f:
        result = pickle.load(f)
        result = process_result(result)
        result_list.append(result)

In [96]:
tmp = torch.stack(result_list)
tmp = torch.sum(tmp, dim=[0])

tmp = torch.argmax(tmp, axis=1)



tensor([17, 14, 11,  ..., 17,  2, 17])

In [97]:
preds = np.array([label_decoder[int(val)] for val in tmp.numpy()])

In [100]:
submission = pd.read_csv(f'data/sample_submission.csv')
submission['label'] = preds

contributors = ['_'.join(os.path.basename(path).split("_")[:-1]) for path in glob("results/*")]
save_file_path = os.path.join('submissions', 'ensemble',','.join(contributors))  + '-submission.csv'

submission.to_csv(save_file_path, index=False)